In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from collections import Counter

## Теоретическая справка

### Критерий Пирсона

$$\hat{r}_{XY} = \frac{\hat{k}_{XY}(n)}{\bar{s}_X \bar{s}_Y}$$

где

$$\hat{k}_{XY}(n) = \frac{1}{n} \sum_{i=1}^n X_iY_i - \overline{X}_n \overline{Y}_n$$

$$\bar{s}_n^2 = \frac{1}{n} \sum_{i=1}^n (X_i - \overline{X}_n)^2$$

Тогда статистика

$$T = \frac{\sqrt{n - 2}\hat{r}_{XY}}{\sqrt{1 - \hat{r}_{XY}^2}}$$

T имеет распределение Стьюдента с $n - 2$ степенями свободы.

### Критерий Кендалла

Если нет связок:

$$\hat{\tau} = \frac{2 \sum\limits_{i\leq i < j \leq n} \mathrm{sign}\{(X_i - X_j)(Y_i -  Y_j)\}}{n(n-1)}$$

Если есть связки:

$$\hat{\tau} = \frac{\sum\limits_{i\leq i < j \leq n} \mathrm{sign}\{(X_i - X_j)(Y_i -  Y_j)\}}{\sqrt{(\frac{n(n-1)}{2} - u_1)(\frac{n(n-1)}{2} - u_2)}}$$

где

$$u_k = \frac1 2 \sum_{i=1}^q u_{ki}^2 - u_{ki}$$

$q$ - количество связок в выборке $k$ ($X$ или $Y$), $u_{ki}$ - размер этой связки

Тогда 

$$T = \frac{\hat{\tau}}{\sqrt D} \sim \frac{3\sqrt n \hat{\tau}}{2}$$

где 

$$D = \frac{4n+10}{9n(n-1)}$$


T распределена нормально

### Критерий Спирмена

$$\hat{\rho} = \frac{\sum\limits_{i=1}^n (R_i - \overline R)(S_i - \overline S)}{\sqrt{\sum\limits_{i=1}^n (R_i - \overline R)^2 \times \sum\limits_{i=1}^n (S_i - \overline S)^2}}$$

где $R_i$ - ранг элемента $X_i$ в выборке $X_n$, $S_i$ - ранг элемента $Y_i$ в выборке $Y_n$, а $\overline R$ и $\overline S$ - средние арифметические рангов

тогда $$T = \sqrt{n-1}\hat{\rho}$$

T распределена нормально

In [2]:
tasks = [6, 19, 21]
statisctics = []
p_values = []
h = []

### Задача 6.

Поскольку данные распределены нормально и не являются константными ($D\neq0$), то воспользуемся критерием Пирсона для проверки гипотезы о независимости данных. 

Основная гипотеза:
$H_0:$ данные независимы $\sim r = 0$

Альтернативная гипотеза:
$H_3:$ данные зависимы $\sim r \neq 0$

In [3]:
X = np.array([1.05, 1.06, 0.74, 0.88, 0.92, 1.17, 1.07, 1.12, 1.02, 1.02,
              0.65, 1.25, 1.01, 0.69, 1.13])
Y = np.array([346.2, 378.3, 330.3, 238.8, 331.1, 431.0, 408.3, 383.5, 341.4,
             352.8, 338.1, 446.9, 326.6, 274.7, 381.0])
n = 15
alpha = 0.05

#### Pearson correlation test

In [4]:
k = np.sum(X * Y) / n - X.mean() * Y.mean() # Выборочная ковариация
r = k / np.sqrt(np.var(X) * np.var(Y)) # Выборочный коэффициент корреляции (статистика)
T = np.sqrt(n - 2) * r / np.sqrt(1 - r ** 2) # Статистика с распределением стьюдента
p_value = 2 * min(stats.t.cdf(T, n - 2), 1 - stats.t.cdf(T, n - 2))
p_values.append(p_value)
statisctics.append(r)
assert(np.all(np.isclose(stats.pearsonr(X, Y), (r, p_value))))

In [5]:
if p_value >= alpha:
    print(f'p_value = {p_value} > 0.05 => остаемся в Н0 (выборки независимы)')
    h.append(0)
else:
    print(f'p_value = {p_value} < 0.05 => отвергаем Н0 (выборки зависимы)')
    h.append(3)

p_value = 0.0016793308942941731 < 0.05 => отвергаем Н0 (выборки зависимы)


### Задача 19.

В задаче явно не указано какой критерий использовать. Поскольку баллы за лабораторную работу по сути своей представляют некоторый проранжированный список, то воспользуемся непараметрическим критерием. Я отдам предпочтение критерию Спирмена. Требуется проверить независимость (статистическую) случайных величин. Поэтому

Основная гипотеза:
$H_0:$ СВ X и Y независимы $\sim r = 0$

Альтернативная гипотеза:
$H_3:$ СВ X и Y зависимы $\sim r \neq 0$

In [6]:
X = np.array([10, 13, 5, 18, 19, 20, 20, 16, 15, 14, 16, 15, 20, 17, 14, 10])
Y = np.array([20, 20, 15, 15, 0, 20, 20, 12, 20, 20, 9, 20, 20, 20, 15, 20])
n = X.size
alpha = 0.05

#### Spearman test

In [7]:
R = stats.rankdata(X)
S = stats.rankdata(Y)
r = np.sum((R - R.mean()) * (S - S.mean())) / np.sqrt(np.sum((R - R.mean()) ** 2) * np.sum((S - S.mean()) ** 2))
if (n > 50):
    T = np.sqrt(n - 1) * r # нормальная апроксимация
    p_value = 2 * min(stats.norm.cdf(T), 1 - stats.norm.cdf(T))
else:
    T = np.sqrt(n - 2) * r / np.sqrt(1 - r ** 2) # апроксимация стьюдента
    p_value = 2 * min(stats.t.cdf(T, n - 2), 1 - stats.t.cdf(T, n - 2))
p_values.append(p_value)
statisctics.append(r)
assert(np.all(np.isclose(stats.spearmanr(X, Y), (r, p_value))))
if p_value >= alpha:
    print(f'p_value = {p_value} > 0.05 => остаемся в Н0 (СВ независимы)')
    h.append(0)
else:
    print(f'p_value = {p_value} < 0.05 => отвергаем Н0 (СВ зависимы)')
    h.append(3)

p_value = 0.8041065958627895 > 0.05 => остаемся в Н0 (СВ независимы)


### Задача 21.

Поскольку в задаче явно указано использовать критерий Кендалла, то воспользуемся им. Требуется проверить независимость, поэтому введем следующие гипотезы.

Основная гипотеза:
$H_0:$ параметры независимы $\sim r = 0$

Альтернативная гипотеза:
$H_3:$ параметры зависимы $\sim r \neq 0$

#### Kendall test

In [8]:
X = np.array([13, 7, 10, 9, 9, 13, 8, 9, 5, 24, 12, 8, 3, 6, 3, 18,
             16, 16, 14, 3, 14, 8, 5, 7, 10, 11, 4, 31, 12, 9, 6,
             12, 4, 10, 5, 11, 13, 7, 17, 8, 9, 10, 6, 5, 6, 4,
             11, 11, 3, 7, 8, 11, 9, 11, 8, 7, 9, 6, 9, 9, 11, 11, 5, 8, 4])
Y = np.array([62.4, 69.3, 63.3, 67.8, 69.3, 68.4, 66.3, 60, 66.3, 78, 72.6,
             60.9, 61.5, 70.8, 62.1, 65.7, 62.1, 72.6, 70.5, 66.6, 63.3,
             72.9, 72.9, 58.2, 68.1, 67.8, 63.3, 72.9, 67.2, 64.8, 
             68.7, 68.4, 64.5, 74.1, 71.7, 67.8, 68.1, 25.1, 78.9,
             64.5, 57.6, 71.7, 52.2, 59.7, 48, 62.1, 81.3, 65.1, 63.9, 54.9,
             54.3, 54.6, 62.7, 62.7, 63.9, 64.2, 66.3, 65.4, 67.2, 69,
             56.7, 77.7, 57, 61.5, 65.7])
n = X.size

In [9]:
def calc_u(X):
    u = 0
    for _, count in Counter(X).most_common():
        u += count ** 2 - count
    return u / 2

u_1 = calc_u(stats.rankdata(X))
u_2 = calc_u(stats.rankdata(Y))
tau = 0
for i in range(n-1):
    for j in range(i+1, n):
        tau += np.sign((X[i] - X[j]) * (Y[i] - Y[j]))
tau /= np.sqrt(n * (n-1) / 2 - u_1) * np.sqrt(n * (n-1) / 2 - u_2)
D = (4 * n + 10) / (9 * n * (n - 1))
T = tau / np.sqrt(D)
p_value = 2 * min(stats.norm.cdf(T), 1 - stats.norm.cdf(T))
p_values.append(p_value)
statisctics.append(tau)
if p_value >= alpha:
    print(f'p_value = {p_value} > 0.05 => остаемся в Н0 (параметры независимы)')
    h.append(0)
else:
    print(f'p_value = {p_value} < 0.05 => отвергаем Н0 (параметры зависимы)')
    h.append(3)

p_value = 0.0024220496764277843 < 0.05 => отвергаем Н0 (параметры зависимы)


In [10]:
assert(np.isclose(tau, stats.kendalltau(X, Y)[0]))
p_value, stats.kendalltau(X, Y)[1]

(0.0024220496764277843, 0.0034475221032775885)

Неудивительно, что p_value, посчитанное ручками, отличается от библиотечной, т.к. мы используем нормальную апроксимацию в предположении справедливости $H_0$, а т.к. она несправедлива, то и значение p_value неточное.

In [11]:
answer = pd.DataFrame()
answer['task'] = tasks
answer['stat'] = statisctics
answer['p'] = p_values
answer['h'] = h

In [12]:
answer.to_csv('lab4_answer.csv', index=False)

check all is written ok

In [13]:
pd.read_csv('lab4_answer.csv')

,task,stat,p,h
0,6,0.738096,0.001679,3
1,19,-0.067407,0.804107,0
2,21,0.257557,0.002422,3
